In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
from fractions import Fraction
import warnings
warnings.filterwarnings('ignore')


In [2]:
# parameters
customer_id = '5'
formatted_attribute = 'Cooktop Compatibility'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Window Blinds"""
buckets = """Multiple_Bucket"""

regex_pattern=r'(?i)(halogen)|(ceramic)()'

value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [4]:
# df = query_from_file(file_name='Curate_wipe/BusinessIntelligence/Platform Queries/Bucket Attribute Values for All active Strategies.sql', params=params)
df = query_from_file(file_name='query/uncurated.sql', params=params)
df

product_id external_id  \
0    1185400202    69609896   
1     988762772    69609567   
2    1185400203    69609894   
3    1213267464    69609904   
4     817181653    61019775   
..          ...         ...   
121  1184011997    66717515   
122   817154007    12402783   
123   817196460    40889474   
124   817164556    43267903   
125   988369545    64329406   

                                                                           product_name  \
0         Le Creuset® Signature 7.5 qt. Cast Iron Covered Round Dutch Oven in Deep Teal   
1                            Le Creuset® Signature 3.5 qt. Covered Sauteuse in Licorice   
2         Le Creuset® Signature 4.5 qt. Cast Iron Covered Round Dutch Oven in Deep Teal   
3    Le Creuset® Signature 10.25-Inch Cast Iron Skillet with Helper Handle in Deep Teal   
4                    Anolon® Nouvelle Copper Stainless Steel French Skillets (Set of 2)   
..                                                                                  ...   
121            Farberware® Glide™ 10-Inch Nonstick Copper Ceramic Deep Skillet in Black   
122                                            Royal Albert Old Country Roses Coffeepot   
123                                             Demeyere John Pawson Fry Pan - 9.4-Inch   
124                                   Zwilling J.A. Henckels Aurora 8-Inch Open Fry Pan   
125           Berndes® Vario® Click Induction Plus Nonstick 13-Inch Frying Pan in Black   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              long_desc  \
0                                                                                                                                                                                                                                                       The Signature Round Dutch Oven from Le Creuset features durable and thermally efficient enameled cast iron construction to deliver even heat distribution and excellent heat retention no matter what heat source you use, including induction. Beautiful French-made design moves from oven or stovetop cooking to table presentation and serving seamlessly Enameled cast iron delivers even heat distribution plus excellent heat retention on any heat source Light-colored smooth interior enamel is engineered to resist staining and makes it easy to monitor food as it cooks to help prevent burning and sticking Durable enameled exterior is shock-resistant to prevent chipping and cracking Suitable for use on all stove types, including induction Included tight-fitting lid with an easy-grip, signature handle locks in heat and moisture for tender, flavorful results Wide loop handles provide a secure grip and even weight distribution especially when cookware is full   
1                                                                                             The Signature Sauteuse 

In [5]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))


In [6]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

In [7]:
df['matches'].explode().value_counts()

ceramic    21
halogen     4
Ceramic     2
Name: matches, dtype: int64

In [8]:
df[df['matches'].astype(str)=='ceramic']

Empty DataFrame
Columns: [product_id, external_id, product_name, long_desc, customer_id, attributes, buckets, values, matches]
Index: []

In [5]:
# error

In [6]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['Adhesive Mount',
 'Ceiling Mount',
 'Concealed Mount',
 'Corner Mount',
 'Cupboard Mount',
 'Door Mount',
 'Flush Mount',
 'Frame Mount',
 'Freestanding',
 'Hanging',
 'Hinge Mount',
 'Inside Mount',
 'Leaning',
 'Magnetic Mount',
 'Outside Mount',
 'Pole Mount',
 'Post Mount',
 'Pre-Drilled Mounting Holes',
 'Recessed Mount',
 'Roof Mount',
 'Screw Fittings',
 'Semi-Flush Mount',
 'Suction Mount',
 'Tension Mount',
 'Under Cabinet Mount',
 'Wall Mount']

In [7]:
kwp=KeywordProcessor()
print(kwp.add_keywords_from_list(list_values))
# Extract Key Words
df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

None


0                     []
1                     []
2        [Outside Mount]
3        [Outside Mount]
4                     []
              ...       
25377                 []
25378                 []
25379          [Hanging]
25380                 []
25381                 []
Name: long_desc, Length: 25382, dtype: object

In [8]:
df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
df=df[df['matches'].astype(str)!='[]']
df['matches'].explode().value_counts()

Hanging                       3240
Wall Mount                    2408
Outside Mount                 1455
Freestanding                   285
Ceiling Mount                   75
Roof Mount                      68
Flush Mount                     49
Leaning                         42
Door Mount                      29
Post Mount                      25
Semi-Flush Mount                20
Recessed Mount                   9
Adhesive Mount                   6
Pre-Drilled Mounting Holes       1
Pole Mount                       1
Name: matches, dtype: int64

In [9]:
# error

In [10]:
matches=df
matches['multi_matches']=matches['matches']
# matches['matches']=matches['Max'].astype(str).apply(lambda quote: quote.replace('.0', ''))
matches.head(1)

product_id external_id  \
14   817331667    65220986   

                                                                product_name  \
14  Bel Air Lighting Argyle 15-Inch Post Mount Outdoor Lantern in Black/Gold   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            long_desc  \
14  Inspired by 18th century carriage lanterns, Bel Air Lighting's Argyle Post Mount Outdoor Lantern captures a vintage look for your traditional home. Its tapered glass panes are topped with a curved, beveled roof and finial for added appeal. Traditional carriage lantern design Hexagonal shape with curved roof and finial Beveled clear glass panes Suitable for dry and damp locations Post mount design Must be hardwired Uses one 60-watt incandescent or CFL or LED equivalent bulb (not included) Steel construction Measures 6\" L x 6\" W x 15\" H 1-year manufacturer's warranty Imported Model 4414BG (Black/Gold); 4414WH (White)   

    customer_id attributes        buckets values                   matches  \
14            5       None  Post Lighting   None  [Post Mount, Post Mount]   

               multi_matches  
14  [Post Mount, Post Mount]

In [11]:
matches['matches'].explode().value_counts()

Hanging                       3240
Wall Mount                    2408
Outside Mount                 1455
Freestanding                   285
Ceiling Mount                   75
Roof Mount                      68
Flush Mount                     49
Leaning                         42
Door Mount                      29
Post Mount                      25
Semi-Flush Mount                20
Recessed Mount                   9
Adhesive Mount                   6
Pre-Drilled Mounting Holes       1
Pole Mount                       1
Name: matches, dtype: int64

In [17]:
# # test_list=pd.DataFrame()
# test_list=matches['matches']
# test_list[0:50]

0                                      [Induction]
2                           [Induction, Induction]
3                           [Induction, Induction]
4                                      [Induction]
5                       [Electric, Gas, Induction]
6                                      [Induction]
7                           [Induction, Induction]
8                                      [Induction]
9                           [Induction, Induction]
10                                     [Induction]
11                          [Induction, Induction]
13                          [Induction, Induction]
18                          [Induction, Induction]
19                          [Induction, Induction]
20                          [Induction, Induction]
21                          [Induction, Induction]
22                          [Induction, Induction]
24                                     [Induction]
25                                     [Induction]
26                             

In [12]:
matches['matches']=[list(set(sublist)) for sublist in matches['matches']]
matches

product_id external_id  \
14      817331667    65220986   
20      817446477    69341595   
22      817489661    17745107   
30      817206738    18292432   
58     1211531703    43374564   
...           ...         ...   
25357   817525737    61461550   
25360   817265905    62489669   
25365      478955    43381869   
25370   988457306    69513631   
25373   817542295    63556278   

                                                                   product_name  \
14     Bel Air Lighting Argyle 15-Inch Post Mount Outdoor Lantern in Black/Gold   
20                          MLB Boston Red Sox Fan Favorite Embossed House Flag   
22                                         MLB Texas Rangers Traditions Pennant   
30                                     MLB New York Yankees Cooperstown Pennant   
58             Real Simple® Cordless Cellular 60.5-Inch x 64-Inch Shade in Sand   
...                                                                         ...   
25357                                           Climbing Man Wall Art in Silver   
25360                         Reasons Why for Him 4-Inch x 6-Inch Picture Frame   
25365           Real Simple® Cordless Cellular 41-Inch x 64-Inch Shade in Shale   
25370       O&O by Olivia & Oliver™ 11-Inch x 17-Inch Metal Wall Frame in Black   
25373                   Private Family Beach 22-Inch x 17.81-Inch Wood Wall Art   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              long_desc  \
14                                                                                                                                                                                                                                                                                                                                                                                                                    Inspired by 18th century carriage lanterns, Bel Air Lighting's Argyle Post Mount Outdoor Lantern captures a vintage look for your traditional home. Its tapered glass panes are topped with a curved, beveled roof and finial for added appeal. Traditional carriage lantern design Hexagonal shape with curved roof and finial Beveled clear glass panes Suitable for dry and damp locations Post mount design Must be hardwired Uses one 60-watt incandescent or CFL or LED equivalent bulb (not included) Steel construction Measures 6\" L x 6\" W x 15\" H 1-year manufacturer's warranty Imported Model 4414BG (Black/Gold); 4414WH (White)   
20                                                                                                                                                                                                                                                                                                                                                                                                                                       Proudly display your pro baseball team pride by hanging this eye-catching MLB Fan Favorite Embossed House Flag. Suitab

In [13]:
# matches['matches'][50:100]

In [14]:
matches[curation_col] = matches['matches'].astype(str)
matches

product_id external_id  \
14      817331667    65220986   
20      817446477    69341595   
22      817489661    17745107   
30      817206738    18292432   
58     1211531703    43374564   
...           ...         ...   
25357   817525737    61461550   
25360   817265905    62489669   
25365      478955    43381869   
25370   988457306    69513631   
25373   817542295    63556278   

                                                                   product_name  \
14     Bel Air Lighting Argyle 15-Inch Post Mount Outdoor Lantern in Black/Gold   
20                          MLB Boston Red Sox Fan Favorite Embossed House Flag   
22                                         MLB Texas Rangers Traditions Pennant   
30                                     MLB New York Yankees Cooperstown Pennant   
58             Real Simple® Cordless Cellular 60.5-Inch x 64-Inch Shade in Sand   
...                                                                         ...   
25357                                           Climbing Man Wall Art in Silver   
25360                         Reasons Why for Him 4-Inch x 6-Inch Picture Frame   
25365           Real Simple® Cordless Cellular 41-Inch x 64-Inch Shade in Shale   
25370       O&O by Olivia & Oliver™ 11-Inch x 17-Inch Metal Wall Frame in Black   
25373                   Private Family Beach 22-Inch x 17.81-Inch Wood Wall Art   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              long_desc  \
14                                                                                                                                                                                                                                                                                                                                                                                                                    Inspired by 18th century carriage lanterns, Bel Air Lighting's Argyle Post Mount Outdoor Lantern captures a vintage look for your traditional home. Its tapered glass panes are topped with a curved, beveled roof and finial for added appeal. Traditional carriage lantern design Hexagonal shape with curved roof and finial Beveled clear glass panes Suitable for dry and damp locations Post mount design Must be hardwired Uses one 60-watt incandescent or CFL or LED equivalent bulb (not included) Steel construction Measures 6\" L x 6\" W x 15\" H 1-year manufacturer's warranty Imported Model 4414BG (Black/Gold); 4414WH (White)   
20                                                                                                                                                                                                                                                                                                                                                                                                                                       Proudly display your pro baseball team pride by hanging this eye-catching MLB Fan Favorite Embossed House Flag. Suitab

In [15]:
# matches

In [29]:
# error

In [18]:
# b_set = set(map(tuple,test_list))  #need to convert the inner lists to tuples so they are hashable
# b = map(list,b_set) #Now convert tuples back into lists (maybe unnecessary?)

In [37]:
# b_set

In [38]:
# def rem_dupes(dup_list): 
#     yooneeks = [] 
#     for elem in dup_list: 
#         if elem not in yooneeks: 
#             yooneeks.append(elem) 
#     return yooneeks

In [39]:
# rem_dupes(test_list[0:5])
# test_list[0:50]

In [40]:
# [list(set(sublist)) for sublist in test_list]

In [41]:
# test_list=test_list.reset_index()
# del test_list['index']
# test_list

In [42]:
# test_list[69]

In [43]:
# sorted(set(test_list))

In [44]:
# new_df=pd.DataFrame()
# for i in range(1):
#     new_df.append(sorted(set(test_list[i])))
# new_df

In [45]:
# test_list=lambda x: [list(set(test_list[i])) for i in test_list]
# test_list

In [46]:
# matches['matches'].astype(str).apply(lambda x: list(set(test_list[i])) for i in range(len(test_list)))

In [47]:
# error

In [48]:
# matches[curation_col] = matches['Max'].astype(str)+' in'
# # matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [49]:
## KWP
# matches[curation_col]=matches['matches']
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [ ]:
error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [16]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [17]:
looks_good('Bed Bath & Beyond', attribute, df, matches)